In [1]:
import pandas as pd
from kan.custom_processing import (plot_data_per_interval,
                                   plot_activation_and_spline_coefficients, get_masks)
import matplotlib.pyplot as plt
import os
import datetime
import json
import numpy as np
plt.rcParams['figure.figsize'] = (15, 8)
plt.rcParams['figure.dpi'] = 75
plt.rcParams['savefig.dpi'] = 75*10

root_dir = os.path.join(os.getcwd())
save_dir = os.path.join(root_dir, "custom_figures")
time_stamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')

ground_truth = lambda x0, x1: np.sin(2*x0) + 5*x1

import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from kan.experiments.multkan_hparam_sweep_materials import evaluate_params, _to_tensor, _build_dataset
from kan.utils import ex_round

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"This script is running on {device}.")

x1_grid = np.linspace(-np.pi, np.pi, 60)
x2_grid = np.linspace(-1, 1, 30)

x1, x2= np.meshgrid(x1_grid, x2_grid)
X = np.stack((x1.flatten(), x2.flatten()), axis=1)

y_mesh = ground_truth(x1, x2)
y = y_mesh.flatten().reshape(-1, 1)

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)  # 0.2 × 0.8 = 0.16 (전체의 16%)

scaler_X = MinMaxScaler(feature_range=(0.1, 0.9))
scaler_y = MinMaxScaler(feature_range=(0.1, 0.9))
X_train_norm = scaler_X.fit_transform(X_train)
y_train_norm = scaler_y.fit_transform(y_train)
X_val_norm = scaler_X.transform(X_val)
X_test_norm = scaler_X.transform(X_test)
y_val_norm = scaler_y.transform(y_val)
y_test_norm = scaler_y.transform(y_test)


This script is running on cpu.


In [2]:
from SALib.analyze.sobol import analyze

X_norm = np.concatenate([X_train_norm, X_val_norm, X_test_norm])
y_norm = np.concatenate([y_train_norm, y_val_norm, y_test_norm])

nx = X_norm.shape[1]
problem = {
    'num_vars': nx,
    'names': [f'x{i+1}' for i in range(nx)],
    'bounds': [[0.1, 0.9]] * nx
}

Si = analyze(problem, y_norm.reshape([-1]), print_to_console=True)


          ST   ST_conf
x1  0.974101  0.138121
x2  1.080693  0.141988
          S1   S1_conf
x1  0.089537  0.156158
x2 -0.013932  0.154908
                S2  S2_conf
[x1, x2] -0.091881  0.21938
[ 0.08953655 -0.01393186]


x1 ~ x2.
그런데 negative S1 이 나온다.. (데이터가 부족하거나 너무 x2에 대한 dependency가 떨어져서 그런 걸수도 있음).
하지만 이런 상태에서도 학습은 가능했기 때문에 ST만 사용하면 될듯.